In [ ]:
!pip install tf-nightly-2.0-preview==2.0.0.dev20190606  #tensorflow 2.0
!pip install earthengine-api==0.1.175 #earthengine API
#Load the TensorBoard notebook extension
%load_ext tensorboard

ERROR: Could not find a version that satisfies the requirement tf-nightly-2.0-preview==2.0.0.dev20190606 (from versions: none)
ERROR: No matching distribution found for tf-nightly-2.0-preview==2.0.0.dev20190606
     |████████████████████████████████| 122kB 2.8MB/s 
     |████████████████████████████████| 61kB 8.6MB/s 
     |████████████████████████████████| 2.7MB 44.1MB/s 
  Created wheel for earthengine-api: filename=earthengine_api-0.1.175-cp36-none-any.whl size=144181 sha256=129cf6a3f5a7c5ee1770d97f0171666102304f4d33b497f6b187ee049d51ab1f
  Stored in directory: /root/.cache/pip/wheels/ee/b9/f9/7278e3dd64c0520f3658c9404ade74ab2a06b9b104e6de16f6
Successfully built earthengine-api
  Found existing installation: earthengine-api 0.1.218
    Uninstalling earthengine-api-0.1.218:
      Successfully uninstalled earthengine-api-0.1.218


In [ ]:
# authenticate google cloud account

from google.colab import auth
auth.authenticate_user()

In [ ]:
# authenticate google earth engine account

!earthengine authenticate


Opening the following address in a web browser:

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code

Please authorize access to your Earth Engine account, and paste the generated code below. If the web browser does not start, please manually browse the URL above.

Please enter authorization code: 4/zAE0r_ZXJruo338PbVFnjI6u1f5J113-Ntf5_jBjuk6G4pHJki_Yruk

Successfully saved authorization token.


In [ ]:
# Earth Engine Python API
import ee 
ee.Initialize()

import tensorflow as tf
print('Tensorflow version: ' + tf.__version__)

import folium
print('Folium version: ' + folium.__version__)

# Define the URL format used for Earth Engine generated map tiles.
EE_TILES = 'https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}'

Tensorflow version: 2.2.0-rc3
Folium version: 0.8.3


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
! pip install bqplot
!pip install ipyleaflet
!jupyter nbextension enable --py --sys-prefix ipyleaflet


     |████████████████████████████████| 4.7MB 2.8MB/s 
     |████████████████████████████████| 4.4MB 2.8MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: branca 0.4.0
    Uninstalling branca-0.4.0:
      Successfully uninstalled branca-0.4.0


Traceback (most recent call last):
  File "/usr/local/bin/jupyter-nbextension", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python2.7/dist-packages/jupyter_core/application.py", line 267, in launch_instance
    return super(JupyterApp, cls).launch_instance(argv=argv, **kwargs)
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python2.7/dist-packages/notebook/nbextensions.py", line 988, in start
    super(NBExtensionApp, self).start()
  File "/usr/local/lib/python2.7/dist-packages/jupyter_core/application.py", line 256, in start
    self.subapp.start()
  File "/usr/local/lib/python2.7/dist-packages/notebook/nbextensions.py", line 896, in start
    self.toggle_nbextension_python(self.extra_args[0])
  File "/usr/local/lib/python2.7/dist-packages/notebook/nbextensions.py", line 872, in toggle_nbextension_python
    logger=self.log)
  File "/usr/local/lib/python2.7/dist-pac

In [ ]:
import bqplot
import datetime
import dateutil.parser
import ee
import ipywidgets
import ipyleaflet
import IPython.display
import numpy as np
import pprint
import pandas as pd
import traitlets

**Basics**

In [ ]:
img = ee.Image("USDA/NAIP/DOQQ/m_3210236_se_13_060_20181026").clip(ee.Geometry.Rectangle(-102.59355, 32.37884, -102.50318, 32.4216))

In [ ]:
img.bandNames().getInfo()

['R', 'G', 'B', 'N']

In [ ]:
def GetTileLayerUrl(ee_image_object):
  map_id = ee.Image(ee_image_object).getMapId()
  tile_url_template = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}"
  return tile_url_template.format(**map_id)

In [ ]:
img_tile = GetTileLayerUrl(img)

img_tile

'https://earthengine.googleapis.com/map/88d9220985e94ab522daa01a9e9960b0/{z}/{x}/{y}?token=feed1c5ae07a376426b81addf792cb0b'

**Get Training Dataset from GEE**

In [ ]:
import ee

ee.Initialize()

In [ ]:
def Mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
    '''
    :param center: Center of the map (Latitude and Longitude).
    :param dicc: Earth Engine Geometries or Tiles dictionary
    :param Tiles: Mapbox Bright,Mapbox Control Room,Stamen Terrain,Stamen Toner,stamenwatercolor,cartodbpositron.
    :zoom_start: Initial zoom level for the map.
    :return: A folium.Map object.
    '''
    mapViz = folium.Map(location=center,tiles=Tiles, zoom_start=zoom_start)
    for k,v in dicc.items():
      if ee.image.Image in [type(x) for x in v.values()]:
        folium.TileLayer(
            tiles = v["tile_fetcher"].url_format,
            attr  = 'Google Earth Engine',
            overlay =True,
            name  = k
          ).add_to(mapViz)
      else:
        folium.GeoJson(
        data = v,
        name = k
          ).add_to(mapViz)
    mapViz.add_child(folium.LayerControl())
    return mapViz

In [ ]:
#Mapdisplay([32,-102],img.)

In [ ]:
# 2.4.1 Prediction Area
xmin,ymin,xmax,ymax = [-102.59355, 32.37884, -102.50318, 32.4216]


# Passing a rectangle (prediction area) to Earth Engine
ROI = ee.Geometry.Rectangle([xmin,ymin,xmax,ymax])
center = ROI.centroid().getInfo()['coordinates']
center.reverse()
Mapdisplay(center,{'Region of Interest':ROI.getInfo()},zoom_start=12)

In [ ]:
# collect n images within for the given filter
img_collection_ROI = ee.ImageCollection(ee.ImageCollection("USDA/NAIP/DOQQ")).filterBounds(ROI).filterDate('2018','2020')


In [ ]:
# number of images collected in the given filter limits

img_collection_ROI.size().getInfo()

3

In [ ]:
#meta data
img_collection_ROI.getInfo()

{'bands': [],
 'features': [{'bands': [{'crs': 'EPSG:26913',
     'crs_transform': [0.6, 0.0, 729012.0, 0.0, -0.6, 3591834.0],
     'data_type': {'max': 255,
      'min': 0,
      'precision': 'int',
      'type': 'PixelType'},
     'dimensions': [10580, 12290],
     'id': 'R'},
    {'crs': 'EPSG:26913',
     'crs_transform': [0.6, 0.0, 729012.0, 0.0, -0.6, 3591834.0],
     'data_type': {'max': 255,
      'min': 0,
      'precision': 'int',
      'type': 'PixelType'},
     'dimensions': [10580, 12290],
     'id': 'G'},
    {'crs': 'EPSG:26913',
     'crs_transform': [0.6, 0.0, 729012.0, 0.0, -0.6, 3591834.0],
     'data_type': {'max': 255,
      'min': 0,
      'precision': 'int',
      'type': 'PixelType'},
     'dimensions': [10580, 12290],
     'id': 'B'},
    {'crs': 'EPSG:26913',
     'crs_transform': [0.6, 0.0, 729012.0, 0.0, -0.6, 3591834.0],
     'data_type': {'max': 255,
      'min': 0,
      'precision': 'int',
      'type': 'PixelType'},
     'dimensions': [10580, 12290],
  

In [ ]:
# how many images in the given area in the given data range 

print(img_collection_ROI.size().getInfo())

3


In [ ]:
# bands in the first image of the imageCollection
img_collection_ROI.first().bandNames().getInfo()

['R', 'G', 'B', 'N']

In [ ]:
# select an image and clip to the region of interest

img = ee.Image("USDA/NAIP/DOQQ/m_3110101_ne_14_060_20181113_20190504").clip(ROI)

In [ ]:
composite = img_collection_ROI.max()
Mapdisplay([32,-102],{'max value composite':composite.getMapId()},zoom_start=13)

In [ ]:
# just one image

img = ee.Image("USDA/NAIP/DOQQ/m_3110101_ne_14_060_20181113_20190504").clip(ROI)

In [ ]:
#img_collection_ROI.filterMetadata('IMAGE_QUALITY','equals',9)

In [ ]:
print(img.getInfo())

{'type': 'Image', 'bands': [{'id': 'R', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'crs': 'EPSG:26914', 'crs_transform': [0.6, 0.0, 222144.0, 0.0, -0.6, 3544362.0]}, {'id': 'G', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'crs': 'EPSG:26914', 'crs_transform': [0.6, 0.0, 222144.0, 0.0, -0.6, 3544362.0]}, {'id': 'B', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'crs': 'EPSG:26914', 'crs_transform': [0.6, 0.0, 222144.0, 0.0, -0.6, 3544362.0]}, {'id': 'N', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'crs': 'EPSG:26914', 'crs_transform': [0.6, 0.0, 222144.0, 0.0, -0.6, 3544362.0]}], 'version': 1585607051808997, 'id': 'USDA/NAIP/DOQQ/m_3110101_ne_14_060_20181113_20190504', 'properties': {'system:footprint': {'type': 'MultiPoint', 'coordinates': []}, 'system:time_start': 1542067200000, 'system:time_end': 1542067200000, 'system:asset_size': 532244936, 'system:

In [ ]:
prediction_area.first().bandNames().getInfo()

['R', 'G', 'B', 'N']

In [ ]:
import folium

Peru = ee.Image("USDA/NAIP/DOQQ/m_3110101_ne_14_060_20181113_20190504")\
         .clip(ROI)

# Define the URL format used for Earth Engine generated map tiles.
EE_TILES = 'https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}'

# Use folium to visualize the imagery.
mapid = Peru.getMapId({'pallete': ['black', 'white'], 'min': 0, 'max': 3000})

map = folium.Map(location=[32, -102],zoom_start=5)
folium.TileLayer(
    
    tiles = mapid["tile_fetcher"].url_format,
    attr='Google Earth Engine',
    overlay=True,
    name='median composite',
  ).add_to(map)
map.add_child(folium.LayerControl())
map